# Project 1: First steps in Machine Learning (40 Points)
In this project, you will train and test your first machine learning models. Please follow the **TODO**s in this notebook. There are practical and theoretical tasks to do.<br>
When working on the tasks please consider the following information:
* write short texts in **full sentences** answering the TODOs. Note, that analyzing the theoratical parts of the project give roughly 2/3 of all points
* when describing classifiers explain at least the training, testing, and the hyper-parameters
* always train your models using the given training sets
* evaluate the models using the given test sets
* have a look at all imports in this notebook; they already define which method you should use

You should work in a group of 3. Please enter your names and your TA here.<br>
Students: Basel Ammo & Linus Behrbohm<br>
TA: Yasir Plückebaum

On Monday, **7th December**, there will be a Q&A session in the tutorials! Start to work on this project from now on and take the offer to resolve any remaining ambiguity.
This assignment is due on **Sunday, 29.11.2020 11:59pm**. Please upload your solution to the Lernraum+.<br>
 For a submission you need to be part of a assignment group.

## kNN - Classifier

In [ ]:
import numpy as np
# load dataset 1
data_set1 = np.load('dataset_1.npz')

### The first dataset

The first dataset is split into a training and a test dataset with input and expected output data each. The format of the input data is a 2D vector, and the output format is an integer with a value of 0 or 1. There are 536 samples in the training data set and 264 samples in the test data set.

<font color='red'>
0,5/1P , Anzahl Datenpunkte pro Klasse vergessen
</font>

In [ ]:
# TODO: take a closer look at the dataset, e.g. number of samples, dimensionality, labels, etc.
import pandas as pd
import matplotlib.pyplot as plt

# generic function to extract named properties from the data set items
# fs is a list of tuples, first a name, then a function mapping an item to a property
def property_table(fs, data_set):
    keys = list(data_set.keys()) # row names
    props = list(map(lambda f: f[0], fs)) # column names
    funcs = list(map(lambda f: f[1], fs)) # property functions
    items = list(map(lambda t: t[1], data_set.items())) # data items
    data = zip(*list(map(lambda f: map(f, items), funcs))) # each prop for each item
    return pd.DataFrame(data, index=keys, columns=props) # use pandas.DataFrame to print a table
    
data_frame = property_table([
        ('Sample Count', lambda item: len(item)),
        ('Shape', lambda item: item.shape),
        ('Type', lambda item: type(item[0])),
        ('First', lambda item: item[0]),
    ], data_set1)

print(data_frame)
def plot_data(data_set):
    X_train = data_set['X_train']
    X_test = data_set['X_test']
    y_train = data_set['y_train']
    y_test = data_set['y_test']
    fig, axs = plt.subplots(1, 2)
    fig.set_figwidth(15)
    axs[0].scatter(*zip(*X_train), c=y_train)
    axs[0].set_title("Training")

    axs[1].scatter(*zip(*X_test), c=y_test)
    axs[1].set_title("Test")

plot_data(data_set1)

<font color='red'>
Sehr sauber! 1(+0,5)P. Muss zugeben, ich dachte kurz der Code wäre aus dem Internet
</font>

### the Model

The kNN-Classifier learns a probability distribution in a space based on a set of input points with previously assigned classes, i.e. by supervised training. There can be any number of classes. The kNN-Classifier calculates the probability of any new point belonging to a specific class as the proportion of points from the k nearest known points with that specific class. I.e. for any given point x, the probability to belong to a specific class c is calculated by the number of points with class c in the nearest k points of x, divided by k. The predicted class by the kNN-Classifier is the class with the maximum probability.

<font color='red'>
    1/3P , K als Hyperparameter und Training vergessen. (In wie weit lernt Knn eine Wsk- Verteilung?)
</font>

In [ ]:
# TODO: train a kNN classifier with k=5 on the training set and test it with the test set

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import functools
foldl = lambda acc, f, xs: functools.reduce(f, xs, acc)

def train_classifier(clf, data_set):
    X_train = data_set['X_train']
    y_train = data_set['y_train']
    clf.fit(X_train, y_train)
    return clf

def test_classifier(clf, data_set):
    X_test = data_set['X_test']
    y_test = data_set['y_test']
    y=clf.predict(X_test)
    # ensure y_test.size != 0
    if y_test.size == 0: return 1
    # count number of correct predictions and divide them by total number of samples
    return foldl(0, lambda acc, t: acc + (t[0]==t[1]), zip(y, y_test))/y_test.size

def train_and_test_classifier(clf, data_set):
    clf = train_classifier(clf, data_set)
    return test_classifier(clf, data_set)

def test_data_set(clf, data_set):
    accuracy = train_and_test_classifier(clf, data_set)
    print(f'Accuracy {accuracy}')
    
test_data_set(KNeighborsClassifier(n_neighbors=5), data_set1)

<font color='red'>
    1P
</font>

The kNN-Classifier with k=5 reached a 98% accuracy on the test data set.

<font color='red'>
1P
</font>

In [ ]:
# TODO: train and test the kNN classifier for different values of k on dataset 1.
#       Plot the accuracy for different values of k. Choose usefull lower and upper bounds for k.
import matplotlib.pyplot as plt
lo = 1
hi = 536
accuracies = list(map(lambda k: train_and_test_classifier(KNeighborsClassifier(n_neighbors=k), data_set1), range(lo,hi)))
plt.plot(accuracies)
max_index = max(range(0, len(accuracies)), key=lambda i: accuracies[i])
print('The best choice for k in [{}, {}] is {} with an accuracy of {}'.format(lo, hi, max_index+lo, accuracies[max_index]))

<font color='red'>1P</font>


**TODO:** Answer the following questions in a **full text**.

* Describe your choice of values of k. Why did you choose them?
* For which values of k does the model perform best?
* Would this value perform best on another dataset as well?
* How can k be choosen?
* What is the smallest and the greatest possible value for k? What would happen if we would choose these special values?

I chose values 1 between 536, because 1 is the minimum number of neighbors possible and 536 is the number of samples in the data set, so that would be the maximum number of neighbors possible in this data set. The model turned out to perform best for k=5. This may be different for other data sets, and it depends on the number of samples in the data set, aswell as the distribution of the classes being learned. This has to be learned for each data set, by trying out different values for k. A rule of thumb might be, if you have a lot of data already, you can choose lower values of k, because the boundaries between the classes will already be well distiguished in the data you already have. Higher values of k are more sensitive to unbalanced data sets, whereas smaller values for k are more sensitive to noisy data sets. As mentioned, the smallest possible value for k is 1, as this is the minimum number of neighbors possible. The largest possible value is the number of points in the data set, as this is the maximum number of neighbors a point can have in any data set. Note that the point being predicted is not part of the data set we already know the classes of. If we chose k=1 we would classify each new point as the same class as the closest known point next to it. If we chose k as the total number of samples in data set, we would predict every new point to be of the class that is most frequent in the data set, since all points are considered.


<font color='red'>
    1,5/2P
    -0,5P "How can k be choosen?" Das wollte auf cross-validation und Bayesian techniques hinaus</font>


# <font color='red'>7,5/10P</font>

## Logistic Regression
Let's try another model as well.

### the Dataset

In [ ]:
# TODO: load dataset_2.npz and analyze the dataset
data_set2 = np.load('dataset_2.npz')

The second dataset is also split into a training and a test data set with input and expected output data each. The format of the input data is a 2D vector, and the output format is an integer with a value of 0 or 1. There are 670 samples in the training data set and 330 samples in the test data set.

<font color='red'>0,5/1P, samples per class fehlt</font>

In [ ]:
data_frame = property_table([
        ('Sample Count', lambda item: len(item)),
        ('Shape', lambda item: item.shape),
        ('Type', lambda item: type(item[0])),
        ('First', lambda item: item[0]),
    ], data_set2)

print(data_frame)

plot_data(data_set2)

<font color='red'>1P</font>


### the Model

The logistic regression model finds a linear decision boundary by applying gradient descent to a loss function of the likelihood of a point belonging to a specific class: `L(P(y=1|x))` This loss function is usually the negative logarithm.

<font color='red'>1/3P, es fehlt training und predicting</font>

In [ ]:
# TODO: train and test logistic regression on dataset_2.npz (using the training and test set),
# plot the dataset and the decision boundary, own implementation needed (see lecture slide 51 in slides1.pdf)

from sklearn.linear_model import LogisticRegression

clf = train_classifier(LogisticRegression(), data_set2)
accuracy = test_classifier(clf, data_set2)
print(accuracy)
def plot_decision_boundary(clf, data_set):

    X_train = data_set['X_train']
    X_test = data_set['X_test']
    y_train = data_set['y_train']
    y_test = data_set['y_test']
    X = np.concatenate((X_train, X_test))
    y = np.concatenate((y_train, y_test))
    xmin, ymin = np.min(X, axis=0)
    xmax, ymax = np.max(X, axis=0)
    b = clf.intercept_[0] # bias
    w1, w2 = clf.coef_.T # weights in W
    
    c = -b/w2 # x axis intercept of decision boundary
    m = -w1/w2 # slope of decision boundary
    print(f'{m} {c}')
    xd = np.array([xmin, xmax])
    yd = m*xd + c # decision boundary
    
    fig, axs = plt.subplots(1, 1)
    fig.set_figwidth(15)
    axs.scatter(*zip(*X), c=y)
    axs.plot(xd, yd, lw=1, ls='--')
    axs.set_title("Decision Boundary")

plot_decision_boundary(clf, data_set2)

<font color='red'>2P</font>


The model converged to a decision boundary with `y = 0.518*x - 0.136`.

<font color='red'>0,5/1P</font>


In [ ]:
# TODO: Check whether w separates the two classes.
#       How does the negative log-likelihood (NLL) change for αw as α goes to infinity?
#
# Hint: Implement an own function which computes the NLL for a new w, and report the NLL for different values.

**TODO:** What can you infer from your observations regarding the training of a model?

### thoeretical considerations

**TODO:** Prove that 

$$\lim_{\alpha\to\infty}-\log\left(P\left(\left.\mathcal{D}\right|\vec{w}\right)\right)=0$$
if $\vec{w}$ splits the data perfectly where $\mathcal{D}$ represents the data.

**Hint:** You can analyze both cases, y=0 and y=1, and find a limit of the likelihood for one sample. Then, calculate the limit of the negative log-likelihood.


<font color='red'>0/12P</font>


# <font color='red'>5/20P</font>

## Comparing kNN and Logistic Regression
Finally, we want you to compare the kNN and the logistic regression classifier.

In [ ]:
# TODO: train both kNN and logistic regression on the training set for both datasets.
#       Evaluate the models using the respective test set.
# Plot the data and the decision boundary of the classifiers. plot_2d_decisionboundary() in utils.py can be used.

from utils import plot_2d_decisionboundary
from utils import plot_classification_dataset

print('### kNN ###')
clf = KNeighborsClassifier(n_neighbors=5)
print('Dataset 1:')
test_data_set(clf, data_set1)
print('Dataset 2:')
test_data_set(clf, data_set2)
print('\n')

print('### Logistic Regression ###')
clf = LogisticRegression()
print('Dataset 1:')
test_data_set(clf, data_set1)
plot_decision_boundary(clf, data_set1)
print('Dataset 2:')
test_data_set(clf, data_set2)
plot_decision_boundary(clf, data_set2)

<font color='red'>3P</font>

The kNN Classifier performs well on both of these data set. The second dataset is easily separable, which also makes it easy for the kNN to find the boundary between the classes. The logistic regression model performs well on the linearly separable data set (data_set2) but performs worse than the kNN on the first data set, because it is not linearly seperable.

<font color='red'>6/7P, "The second dataset is easily separable, which also makes it easy for the kNN to find the boundary between the classes." Warum?</font>

# <font color='red'>9/10P</font>

# <font color='red'>21.5/40P</font>